In [63]:
# 1. Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')


from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report, accuracy_score


In [64]:
#Installing ucimlrepo package
!pip install ucimlrepo

In [65]:
#Fetching UCI repositories 
from ucimlrepo import fetch_ucirepo, list_available_datasets
#List of UCI repositories with ID
list_available_datasets()


-------------------------------------
The following datasets are available:
-------------------------------------
Dataset Name                                                                            ID    
------------                                                                            --    
Abalone                                                                                 1     
Adult                                                                                   2     
Annealing                                                                               3     
Audiology (Standardized)                                                                8     
Auto MPG                                                                                9     
Automobile                                                                              10    
Balance Scale                                                                           12    
Balloons                       

In [66]:
# fetch dataset and create dataframe
adult = fetch_ucirepo(id=2) 
adultDataFrame1 = adult.data.original
adultDataFrame1

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States,<=50K.
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [67]:
originaldataset = adultDataFrame1
originaldataset['workclass'].value_counts()

workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
?                    1836
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64

In [68]:
# adultDataFrame1.replace('?', np.nan, inplace=True)
# # Clean values in income column
# adultDataFrame1['income'].replace('<=50K.', '<=50K',inplace = True)
# adultDataFrame1['income'].replace('>50K.', '>50K',inplace = True)

# print("Duplicate Rows:\n", adultDataFrame1.duplicated().sum())
# duplicates  = adultDataFrame1[adultDataFrame1.duplicated()]
# duplicates
# duplicates[duplicates['fnlwgt']== 308144]
# #Check and drop duplicates
# adult_cleanedData = adultDataFrame1.drop_duplicates(inplace=False)

In [69]:
# 2. Data Cleaning
print("Duplicate Rows:\n", adultDataFrame1.duplicated().sum())
# Identify rows where all values are NaN
# Identify rows where all values are NaN
blank_rows = adultDataFrame1.isnull().all(axis=1)

# Print the count of blank rows
print("Blank Rows (All NaN values):", blank_rows.sum())
# Handling missing values ('?' is treated as missing in this dataset)
adultDataFrame1.replace('?', np.nan, inplace=True)

#Check and drop duplicates
# adult_cleanedData = adultDataFrame1.drop_duplicates(inplace=False)

 # Drop unnecessary columns if needed (e.g., 'fnlwgt' as it doesn't contribute to income prediction)
#adult_cleanedData = adult_cleanedData.copy()

# Now drop the column
#adult_cleanedData.drop(columns=['fnlwgt'], inplace=True)

# Clean values in income column
adultDataFrame1['income'].replace('<=50K.', '<=50K',inplace = True)
adultDataFrame1['income'].replace('>50K.', '>50K',inplace = True)
#adult_cleanedData
adultDataFrame1['income'].value_counts()

Duplicate Rows:
 29
Blank Rows (All NaN values): 0


income
<=50K    37155
>50K     11687
Name: count, dtype: int64

In [70]:
adultDataFrame1['education_new'] = np.where(np.isin(adultDataFrame1.education,['Preschool','1st-4th','5th-6th']),'Elementary',
                           np.where(np.isin(adultDataFrame1.education,['7th-8th']),'Middle-School',
                           np.where(np.isin(adultDataFrame1.education,['9th','10th','11th','12th']),'High-School',adultDataFrame1.education)))
adultDataFrame1[['education_new','education']].value_counts()

education_new  education   
HS-grad        HS-grad         15784
Some-college   Some-college    10878
Bachelors      Bachelors        8025
Masters        Masters          2657
Assoc-voc      Assoc-voc        2061
High-School    11th             1812
Assoc-acdm     Assoc-acdm       1601
High-School    10th             1389
Middle-School  7th-8th           955
Prof-school    Prof-school       834
High-School    9th               756
               12th              657
Doctorate      Doctorate         594
Elementary     5th-6th           509
               1st-4th           247
               Preschool          83
Name: count, dtype: int64

In [71]:
# encoding values 
from sklearn.preprocessing import LabelEncoder
encode_columns = ['workclass','marital-status','occupation','education_new','relationship','race','sex','native-country','income']

# unicode income
#cen_inc['income'] = cen_inc['income'].replace({'>50K':1, '<=50K':0})

for column in encode_columns:
    adultDataFrame1[column+'_enc'] = LabelEncoder().fit_transform(adultDataFrame1[column])

In [51]:
adultDataFrame1['workclass'].isnull().sum()

np.int64(2799)

In [52]:
adultDataFrame1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   age                 48842 non-null  int64 
 1   workclass           46043 non-null  object
 2   fnlwgt              48842 non-null  int64 
 3   education           48842 non-null  object
 4   education-num       48842 non-null  int64 
 5   marital-status      48842 non-null  object
 6   occupation          46033 non-null  object
 7   relationship        48842 non-null  object
 8   race                48842 non-null  object
 9   sex                 48842 non-null  object
 10  capital-gain        48842 non-null  int64 
 11  capital-loss        48842 non-null  int64 
 12  hours-per-week      48842 non-null  int64 
 13  native-country      47985 non-null  object
 14  income              48842 non-null  object
 15  education_new       48842 non-null  object
 16  workclass_enc       48

In [72]:
# Impute numerical columns with median 
num_imputer = SimpleImputer(strategy='median') 
adultDataFrame1[['occupation_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']] = num_imputer.fit_transform(adultDataFrame1[['occupation_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']])

#  ### Part 1: Impute Missing workclass ###

# # Separate rows with missing 'workclass' 
missing_wc = adultDataFrame1[adultDataFrame1['workclass'].isna()] 
not_missing_wc = adultDataFrame1[adultDataFrame1['workclass'].notna()]

# # Label encode 'workclass' for rows without missing values
le_wc = LabelEncoder()
not_missing_wc['wc_Enc'] = le_wc.fit_transform(not_missing_wc['workclass'])

# # Features (excluding 'workplace') and target ('wc_Enc')
X_wc = not_missing_wc[['occupation_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']] 
y_wc = not_missing_wc['wc_Enc']   


# Train Random Forest Classifier for 'workclass' 
rf_classifier_wc = RandomForestClassifier(n_estimators=100, random_state=42) 
rf_classifier_wc.fit(X_wc[['occupation_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']], y_wc) 

# # Predict missing 'workclass' for rows with missing values
X_missing_wc = missing_wc[['occupation_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']] 
missing_wc_pred = rf_classifier_wc.predict(X_missing_wc[['occupation_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']]) 

# Decode predicted 'workclass' back to original categories 
missing_wc['wc_Predicted'] = le_wc.inverse_transform(missing_wc_pred)

# # Fill missing 'workclass' with predicted values 
adultDataFrame1.loc[adultDataFrame1['workclass'].isna(), 'workclass'] = missing_wc['wc_Predicted']


In [73]:
# Impute numerical columns with median 
num_imputer = SimpleImputer(strategy='median') 
adultDataFrame1[['workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']] = num_imputer.fit_transform(adultDataFrame1[['workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']])

#  ### Part 1: Impute Missing occupation ###

# # Separate rows with missing 'occupation' 
missing_wc = adultDataFrame1[adultDataFrame1['occupation'].isna()] 
not_missing_wc = adultDataFrame1[adultDataFrame1['occupation'].notna()]

# # Label encode 'workclass' for rows without missing values
le_wc = LabelEncoder()
not_missing_wc['oc_Enc'] = le_wc.fit_transform(not_missing_wc['occupation'])

# # Features (excluding 'workplace') and target ('wc_Enc')
X_wc = not_missing_wc[['workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']] 
y_wc = not_missing_wc['oc_Enc']   


# Train Random Forest Classifier for 'workclass' 
rf_classifier_wc = RandomForestClassifier(n_estimators=100, random_state=42) 
rf_classifier_wc.fit(X_wc[['workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']], y_wc) 

# # Predict missing 'workclass' for rows with missing values
X_missing_wc = missing_wc[['workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']] 
missing_wc_pred = rf_classifier_wc.predict(X_missing_wc[['workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']]) 

# Decode predicted 'workclass' back to original categories 
missing_wc['oc_Predicted'] = le_wc.inverse_transform(missing_wc_pred)

# # Fill missing 'workclass' with predicted values 
adultDataFrame1.loc[adultDataFrame1['occupation'].isna(), 'occupation'] = missing_wc['oc_Predicted']


In [74]:
adultDataFrame1['occupation'].value_counts()

occupation
Prof-specialty       6706
Exec-managerial      6345
Adm-clerical         6294
Craft-repair         6259
Sales                5535
Other-service        5499
Machine-op-inspct    3027
Transport-moving     2495
Handlers-cleaners    2132
Farming-fishing      1701
Tech-support         1493
Protective-serv      1099
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64

In [ ]:
#original dataset
occupation
Prof-specialty       6167
Craft-repair         6107
Exec-managerial      6084
Adm-clerical         5608
Sales                5504
Other-service        4919
Machine-op-inspct    3019
Transport-moving     2355
Handlers-cleaners    2071
Farming-fishing      1487
Tech-support         1445
Protective-serv       983
Priv-house-serv       240
Armed-Forces           15

#sailaja's initial 
occupation
Craft-repair         6431
Prof-specialty       6428
Exec-managerial      6281
Adm-clerical         6105
Sales                5906
Other-service        5523
Machine-op-inspct    3130
Transport-moving     2450
Handlers-cleaners    2228
Farming-fishing      1587
Tech-support         1467
Protective-serv      1006
Priv-house-serv       256
Armed-Forces           15

#sailaja's latest
Prof-specialty       6436
Craft-repair         6429
Exec-managerial      6289
Adm-clerical         6087
Sales                5906
Other-service        5524
Machine-op-inspct    3140
Transport-moving     2443
Handlers-cleaners    2218
Farming-fishing      1582
Tech-support         1473
Protective-serv      1013
Priv-house-serv       258
Armed-Forces           15

In [25]:
adultDataFrame1['income_enc'].value_counts()

income_enc
0    37155
1    11687
Name: count, dtype: int64

In [75]:
# Impute numerical columns with median 
num_imputer = SimpleImputer(strategy='median') 
adultDataFrame1[['occupation_enc','workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']] = num_imputer.fit_transform(adultDataFrame1[['occupation_enc','workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']])

#  ### Part 1: Impute Missing native-country ###

# # Separate rows with missing 'occupation' 
missing_wc = adultDataFrame1[adultDataFrame1['native-country'].isna()] 
not_missing_wc = adultDataFrame1[adultDataFrame1['native-country'].notna()]

# # Label encode 'workclass' for rows without missing values
le_wc = LabelEncoder()
not_missing_wc['nc_Enc'] = le_wc.fit_transform(not_missing_wc['native-country'])

# # Features (excluding 'workplace') and target ('wc_Enc')
X_wc = not_missing_wc[['occupation_enc','workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']] 
y_wc = not_missing_wc['nc_Enc']   


# Train Random Forest Classifier for 'workclass' 
rf_classifier_wc = RandomForestClassifier(n_estimators=100, random_state=42) 
rf_classifier_wc.fit(X_wc[['occupation_enc','workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']], y_wc) 

# # Predict missing 'workclass' for rows with missing values
X_missing_wc = missing_wc[['occupation_enc','workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']] 
missing_wc_pred = rf_classifier_wc.predict(X_missing_wc[['occupation_enc','workclass_enc','education_new_enc','sex_enc','marital-status_enc','age','relationship_enc','race_enc','hours-per-week','capital-gain','capital-loss','income_enc']]) 

# Decode predicted 'workclass' back to original categories 
missing_wc['nc_Predicted'] = le_wc.inverse_transform(missing_wc_pred)

# # Fill missing 'workclass' with predicted values 
adultDataFrame1.loc[adultDataFrame1['native-country'].isna(), 'native-country'] = missing_wc['nc_Predicted']


In [76]:
adultDataFrame1['native-country'].value_counts()

native-country
United-States                 44596
Mexico                          967
Philippines                     321
Germany                         206
Puerto-Rico                     185
Canada                          182
India                           161
El-Salvador                     156
Cuba                            138
China                           137
England                         127
South                           121
Jamaica                         106
Italy                           105
Dominican-Republic              103
Japan                            96
Vietnam                          89
Guatemala                        88
Poland                           87
Columbia                         85
Haiti                            75
Taiwan                           67
Portugal                         67
Iran                             61
Nicaragua                        49
Greece                           49
Peru                             46
Ecuador      

In [9]:
# #filling the missing values for workclass column

# #identify missing values in workclass column
# test_data = adultDataFrame1[(adultDataFrame1['workclass'].isnull())].copy()
# test_label = test_data.workclass

# train_data = adultDataFrame1[(adultDataFrame1['workclass'].notnull())].copy()
# train_label = train_data.workclass

# #remove the workclass column from train data and test data
# test_data.drop(columns = ['workclass'], inplace = True)
# train_data.drop(columns = ['workclass'], inplace = True)

# # Initialize OneHotEncoder
# encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# # Fit the encoder on the training data
# train_encoded = encoder.fit_transform(train_data)

# # Apply the same encoding to the test data
# test_encoded = encoder.transform(test_data)

# # Ensure both datasets have the same columns
# train_data_encoded = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out())
# test_data_encoded = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out())


# # Perform one-hot encoding on train_data and test_data
# train_data_encoded = pd.get_dummies(train_data)
# test_data_encoded = pd.get_dummies(test_data)

# # Ensure test_data_encoded has the same columns as train_data_encoded
# test_data_encoded = test_data_encoded.reindex(columns=train_data_encoded.columns, fill_value=0)

# #Train and predict the model using logistic regression
# log_reg = LogisticRegression()
# log_reg.fit(train_data_encoded, train_label)
# log_reg_pred = log_reg.predict(test_data_encoded)

# #Train and predict the model using decision tree classifier
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(train_data_encoded, train_label)
# clf_pred = clf.predict(test_data_encoded)

# #Train and predict the model using random forest classifier
# r_forest = RandomForestClassifier(n_estimators=10)
# r_forest.fit(train_data_encoded, train_label)
# r_forest_pred = r_forest.predict(test_data_encoded)

# # Determine the majority class for 'workclass'
# majority_class = adultDataFrame1.workclass.value_counts().index[0]

# # Create DataFrame for predictions from different models
# pred_df = pd.DataFrame({'RFor': r_forest_pred, 'DTree': clf_pred, 'LogReg': log_reg_pred})
# #pred_df

# #Determine the overall prediction using majority voting
# overall_pred = pred_df.apply(lambda x: x.value_counts().index[0] if x.value_counts()[0] > 1 else majority_class, axis=1)
# #overall_pred

# # Ensure the 'overall_pred' has the same index as the rows we want to update
# mask = adultDataFrame1['workclass'].isnull()
# adultDataFrame1.loc[mask, 'workclass'] = overall_pred.values

# # Verify the results
# print(adultDataFrame1.workclass.value_counts())
# print(adultDataFrame1.workclass.unique())

/Users/sravanthithiruveedi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/var/folders/ln/pk1mgs_514n7ydhny2twzb680000gn/T/ipykernel_23432/2941804840.py:58: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  overall_pred = pred_df.apply(lambda x: x.value_counts().index[0] if x.value_counts()[0] > 1 else majority_class, axis=1)
/var/folders/ln/pk1mgs

workclass
Private             36677
Self-emp-not-inc     3875
Local-gov            3140
State-gov            1981
Self-emp-inc         1697
Federal-gov          1434
Without-pay            21
Never-worked           17
Name: count, dtype: int64
['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov'
 'Self-emp-inc' 'Never-worked' 'Without-pay']


In [55]:
# 3. Exploratory Data Analysis (EDA)
# Correlation analysis (only for numeric features)
sns.heatmap(adult_cleanedData.corr(), annot=True, cmap='coolwarm')
plt.show()



ValueError: could not convert string to float: 'State-gov'

In [ ]:
# 4. Data Preprocessing
# Label Encoding for categorical variables
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])

# Normalization/Standardization
scaler = StandardScaler()
df[['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']] = scaler.fit_transform(
    df[['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']]
)

# Splitting features and target
X = df.drop('income', axis=1)
y = df['income']


In [ ]:
# 5. Feature Selection (using Chi-square test for categorical-target relationships)
selector = SelectKBest(chi2, k=10)
X_new = selector.fit_transform(X, y)


In [ ]:
# 6. Model Training Setup
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
# 7. Model Explanation and Execution (RandomForestClassifier with hyperparameter tuning)
pipeline = Pipeline([
    ('classifier', RandomForestClassifier())
])

# Hyperparameter tuning
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1, scoring='accuracy')
grid_search.fit(X_train, y_train)



In [ ]:
# 8. Model Evaluation
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nAccuracy Score: ", accuracy_score(y_test, y_pred))

# Best parameters found by grid search
print("\nBest Hyperparameters:\n", grid_search.best_params_)